In [ ]:
import requests
import pandas as pd

# Define the API endpoint and parameters
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
params = {
    'format': 'geojson',
    'starttime': '2022-01-01',
    'endtime': '2022-12-31',
    'minmagnitude': 5
}

# Send the GET request
response = requests.get(url, params=params)
data = response.json()

# Extract features
features = data['features']

# Create lists to hold data
places = []
magnitudes = []
times = []
latitudes = []
longitudes = []

# Populate lists with data
for feature in features:
    properties = feature['properties']
    geometry = feature['geometry']
    places.append(properties['place'])
    magnitudes.append(properties['mag'])
    times.append(pd.to_datetime(properties['time'], unit='ms'))
    latitudes.append(geometry['coordinates'][1])
    longitudes.append(geometry['coordinates'][0])

# Create a DataFrame
df = pd.DataFrame({
    'Place': places,
    'Magnitude': magnitudes,
    'Time': times,
    'Latitude': latitudes,
    'Longitude': longitudes
})

# Display the first few rows
print(df.head())


In [ ]:
print(df.describe())


In [ ]:
print(df.nlargest(10, 'Magnitude'))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df['Magnitude'], bins=20, edgecolor='black')
plt.title('Distribution of Earthquake Magnitudes in 2022')
plt.xlabel('Magnitude')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import folium
from folium.plugins import HeatMap

# Create a base map
m = folium.Map(location=[0, 0], zoom_start=2)

# Add earthquake locations to the map
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude'],
        popup=f"{row['Place']} (Mag: {row['Magnitude']})",
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(m)

# Display the map
m.save('earthquakes_2022.html')
